In [74]:
#1
import pandas as pd
import numpy as np
import json

import psycopg2
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
import xgboost as xgb
from sklearn import metrics
from xgboost import plot_importance
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from statsmodels.stats.outliers_influence import variance_inflation_factor   
import seaborn as sns
%matplotlib inline
import pickle
import gc
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
plt.style.use('bmh')
from pandas import Series, DataFrame
from sklearn.model_selection import GridSearchCV


import math
import traceback
import sys
import re
import datetime
datetime.timedelta()

datetime.timedelta(0)

In [75]:
#Function for OOT resutls
def OOT_results(xgb_model,OOT):
    t_opt=0.5
    #print("Threshold value is: %.2f" % t_opt)
    
    y_pred = xgb_model.predict_proba(OOT.drop(columns=['Loan_ID','paid_flag']))[:,1]
    fp_r, tp_r, t = metrics.roc_curve(OOT['paid_flag'], y_pred)
    #tp_r, fpr, t = metrics.roc_curve(y_test, y_pred)
    auc = metrics.auc(fp_r, tp_r)
    plt.figure(figsize=(10, 6))
    plt.plot(fp_r, tp_r, label="AUC = %.2f" % auc)
    plt.plot([0,1],[0,1],"r--")
    plt.ylabel("TP rate")
    plt.xlabel("FP rate")
    plt.legend(loc=4)
    plt.title("ROC Curve")
    plt.show()
    
    y_OOT_pred = (xgb_model.predict_proba(OOT.drop(columns=['Loan_ID','paid_flag']))[:,1])
    OOT_pred_label = (xgb_model.predict_proba(OOT.drop(columns=['Loan_ID','paid_flag']))[:,1] >=t_opt).astype(int)
    df_OOT=pd.DataFrame({'Loan_ID':OOT['Loan_ID'],'prob':y_OOT_pred,'pred_label':OOT_pred_label,'actual_label':OOT['paid_flag']})
    df_OOT=df_OOT.sort_values('prob',ascending=False).reset_index()

    total_1=sum(df_OOT['actual_label'])

    print("Results on top 1% data : ")
    df_temp=df_OOT[:math.ceil(len(df_OOT)/100*1)]
    total_sub_1=sum(df_OOT['actual_label'])
    print("precision : ",sum(df_temp['actual_label'])/len(df_temp)*100)
    print("recall : ", sum(df_temp['actual_label'])/sum(df_OOT['actual_label'])*100, "\n")

    print("Results on top 5% data : ")
    df_temp=df_OOT[:math.ceil(len(df_OOT)/100*5)]
    total_sub_1=sum(df_OOT['actual_label'])
    print("precision : ",sum(df_temp['actual_label'])/len(df_temp)*100)
    print("recall : ", sum(df_temp['actual_label'])/sum(df_OOT['actual_label'])*100, "\n")

    print("Results on top 10% data : ")
    df_temp=df_OOT[:math.ceil(len(df_OOT)/100*10)]
    total_sub_1=sum(df_OOT['actual_label'])
    print("precision : ",sum(df_temp['actual_label'])/len(df_temp)*100)
    print("recall : ",sum(df_temp['actual_label'])/sum(df_OOT['actual_label'])*100, "\n")

    print("Results on top 30% data : ")
    df_temp=df_OOT[:math.ceil(len(df_OOT)/100*30)]
    total_sub_1=sum(df_OOT['actual_label'])
    print("precision : ",sum(df_temp['actual_label'])/len(df_temp)*100)
    print("recall : ", sum(df_temp['actual_label'])/sum(df_OOT['actual_label'])*100, "\n")


In [76]:
def convert_predictions_to_bucket(pred):
    if pred <=0.02:
        return 1
    elif pred <=0.1:
        return 2
    elif pred <=1.0:
        return 3

In [77]:
 #creating risk segment
def risk_segment(pred):
        if pred==-1:
            return 'no_risk'
        elif pred==1:
            return 'high_risk'
        elif pred ==2:
            return 'medium_risk'
        elif pred ==3:
            return 'low_risk'

In [11]:
def model_results(xgb_model,ST1,ST2,ST3,paid_file,allocation,comms):
    
    ST2.drop(['L3_index'],axis=1,inplace=True)
    
    
    ST3['Loan_ID']=ST3['Loan_ID'].astype(int)
    ST2['L3_Loan_ID']=ST2['L3_Loan_ID'].astype(int)
    
    df_temp=pd.merge(ST3,ST2,how='left',left_on=['Loan_ID'],right_on=['L3_Loan_ID'])
    df_temp.shape
    df=pd.merge(df_temp,ST1,how='left',left_on=['Loan_ID'],right_on=['Loan_ID'])
    df.shape
    
    OOT=df.copy()
    
    ## Selecting columns based on higher IV values:
    sel_col=[
    'month',
    'Loan_ID',
    'mean_call_duration',
    'call_picked_%',
    'call_response_%',
    'L2_call_duration',
    'L2_call_picked',
    'L2_call_responsive',
    'L2_Promiseto Pay',
    'L1_call_duration',
    'L1_call_picked',
    'L1_call_responsive',
    'L1_Promiseto Pay',
    'L1_RPC',
    'Principle_due_%',
    #'Charges_due_%',
    'Panelty_due_%',
    'Principle_paid_%',
    'paid_flag'    
    ]

    #removing some variables
    sel_col.remove('month')
    sel_col.remove('paid_flag')

    ## Dropping columns with higher VIF value:
    #OOT.drop(['call_picked_%'],axis=1,inplace=True)

    ##Adding 1 feature
    OOT['L2_RPC']=0

    OOT=OOT[sel_col].copy()

    y_pred = xgb_model.predict_proba(OOT.drop(columns=['Loan_ID']))[:,1]

    
    paid_file.shape
    paid_file['Date Repaid'] = pd.to_datetime(paid_file['Date Repaid'], errors='coerce')
    paid_file['day']=paid_file['Date Repaid'].dt.day
    paid_file.shape
    paid_file=paid_file[(paid_file['day']>=0) & (paid_file['day']<=31)]
    paid_file.shape

    #paid_file.drop(['paid_flag'],axis=1,inplace=True)
    paid_file['paid']=1
    paid_file=paid_file.rename(columns={'AgreementNumber':'Loan_ID','paid':'paid_flag'})
    paid_file.drop_duplicates(subset ="Loan_ID", keep = 'first', inplace = True)
    paid_file.shape
    OOT=pd.merge(OOT,paid_file[['Loan_ID','paid_flag']],how='left',left_on='Loan_ID',right_on='Loan_ID')
    OOT.shape
    OOT.columns
    OOT['paid_flag']=OOT['paid_flag'].fillna(0)
    #OOT_results
    OOT_results(xgb_model,OOT)
    
    #Creating june month results
    results=pd.DataFrame({'Loan_ID':list(OOT['Loan_ID']),'Prob':list(y_pred)})
    
    results_updated=results.copy()
   
    df2=results_updated.copy()

    #### Creation of probability buckets

    l=pd.merge(df2,paid_file,how='left',left_on='Loan_ID',right_on='Loan_ID')
    l['paid_flag']=l['paid_flag'].fillna(0)
    l_paid=l[l.paid_flag==1]
    l_paid.shape


    l.loc[:, 'prob_bucket'] = l['Prob'].apply(lambda x: convert_predictions_to_bucket(x))
    l['prob_bucket'].value_counts()

    paid_rate = l.groupby(['prob_bucket'])['paid_flag'].agg(['sum', 'count']).reset_index()
    paid_rate['conversion_rate']=paid_rate['sum']/paid_rate['count']*100
    kk=paid_rate.sort_values('prob_bucket',ascending=False)

    #sum(l['paid_flag']/len(l['Loan_ID']))*100
    
 #   print(l.columns)
 #   print(l.groupby(['prob_bucket']).agg({'Loan_ID':'count'}))

    ## Saving updated ranking

    allocation=allocation[allocation['DPD']>90]
    allocation.shape

    set_A_risk=pd.merge(allocation[['AgreementNumber','OS']],l[['Loan_ID','Prob','prob_bucket']],how='left',left_on='AgreementNumber',right_on='Loan_ID')
    set_A_risk['AgreementNumber']=set_A_risk['AgreementNumber'].astype(int).astype(str)
    
 #   print(set_A_risk.groupby(['prob_bucket']).agg({'AgreementNumber':'count'}))
    
    paid_file['Loan_ID']=paid_file['Loan_ID'].astype(int).astype(str)
    set_A_risk=pd.merge(set_A_risk,paid_file[['Loan_ID','paid_flag']],how='left',left_on='AgreementNumber',right_on='Loan_ID')


#    print(set_A_risk.groupby(['prob_bucket']).agg({'AgreementNumber':'count'}))
    

#     S_AMT=pd.read_excel('RR_jul_S_AMT.xlsx')
#     S_AMT.shape

#     set_A_risk

#     set_A_risk.shape
#     S_AMT['Loan id']=S_AMT['Loan id'].astype(int).astype(str)
#     #june22_paid['Loan_ID']=june22_paid['Loan_ID'].astype(int).astype(str)
#     set_A_risk=pd.merge(set_A_risk,S_AMT,how='left',left_on='AgreementNumber',right_on='Loan id')
#     set_A_risk.shape

    # set_A_risk.drop(['Loan_ID_x','Loan_ID_y'],axis=1,inplace=True)
    set_A_risk['paid_flag']=set_A_risk['paid_flag'].fillna(0)
    set_A_risk['prob_bucket']=set_A_risk['prob_bucket'].fillna(-1)

    set_A_risk.loc[:, 'risk_segment'] = set_A_risk['prob_bucket'].apply(lambda x: risk_segment(x))
    #set_A_risk['risk_segment'].value_counts()
    
 #   print(set_A_risk.groupby(['prob_bucket']).agg({'AgreementNumber':'count'}))
  #  print(set_A_risk.groupby(['risk_segment']).agg({'AgreementNumber':'count'}))
    
    return set_A_risk

